In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
training = pd.read_csv("training.csv")

In [3]:
training.columns

Index(['Unnamed: 0', 'user_id', 'created', 'collected', 'following',
       'followers', 'num_tweets', 'len_name', 'len_bio', 'bot'],
      dtype='object')

In [4]:
training.head()

,Unnamed: 0,user_id,created,collected,following,followers,num_tweets,len_name,len_bio,bot
0,4606,48936207,2009-06-20 00:27:33,2010-03-03 14:11:50,1747,730,57,8,86,1
1,4659,22419996,2009-03-01 17:09:38,2009-11-13 23:46:03,88,24,158,12,14,0
2,4056,43102283,2009-05-28 07:49:30,2010-05-12 04:01:49,2139,2142,493,9,129,1
3,14736,68324565,2009-08-23 23:16:49,2009-11-17 19:00:27,11,32,100,10,0,0
4,2025,15975177,2008-08-24 20:13:13,2009-11-15 14:25:32,348,969,8710,13,151,0


In [5]:
training["bot"].value_counts()

1    22223
0    19276
Name: bot, dtype: int64

In [6]:
x = training.drop(["bot","created","collected", "Unnamed: 0"], axis = 1)
y = training["bot"]

In [7]:
train_x, test_x, train_y, test_y = train_test_split(x,y, test_size=0.2, random_state=0)

In [8]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

num_features=3
lr = LogisticRegression()
feat = RFE(lr, num_features)
feat = feat.fit(train_x,train_y.values.ravel())

C:\Users\famoj\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\famoj\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\famoj\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\famoj\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [9]:
features_selected = list(feat.support_)
pos_features = list(train_x.columns)
selected = [feat for feat,rate in zip(pos_features,features_selected) if rate]

In [10]:
selected

['following', 'num_tweets', 'len_name']

In [11]:
train_y = pd.DataFrame(data=train_y, columns=['bot'])

In [12]:
final_x = train_x[selected]
final_y = train_y['bot']

In [13]:
import statsmodels.api as sm
model=sm.Logit(final_y,final_x)
result=model.fit()
print(result.summary2())

C:\Users\famoj\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


Optimization terminated successfully.
         Current function value: 0.533690
         Iterations 8
                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.227     
Dependent Variable: bot              AIC:              35441.9618
Date:               2019-08-08 09:59 BIC:              35467.1927
No. Observations:   33199            Log-Likelihood:   -17718.   
Df Model:           2                LL-Null:          -22921.   
Df Residuals:       33196            LLR p-value:      0.0000    
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     8.0000                                       
------------------------------------------------------------------
              Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
------------------------------------------------------------------
following     0.0014    0.0000   61.5301  0.0000   0.0014   0.0015
num_tweets   -0.0004    0.0000  -48.6076  0.0000  -0.0004  -0.

In [14]:
#now make the model
regres = LogisticRegression()
regres.fit(final_x, final_y)

C:\Users\famoj\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [15]:
final_test_x = test_x[selected]
predicted_probs=regres.predict(final_test_x)

In [16]:
predicted_probs

array([0, 0, 1, ..., 1, 1, 1], dtype=int64)

In [17]:
from sklearn.metrics import confusion_matrix, classification_report

In [18]:
confusion_matrix(test_y, predicted_probs)

array([[3590,  314],
       [ 868, 3528]], dtype=int64)

In [19]:
len(test_y)

8300

In [20]:
(3590 + 3528)/8300

0.8575903614457832

In [ ]:
#85% accuracy

In [21]:
loye_data = pd.read_csv("Loye_data.csv")

In [22]:
loye_data.columns

Index(['Unnamed: 0', 'following', 'len_name', 'num_tweets', 'screen_name'], dtype='object')

In [24]:
loye_predict = loye_data[selected]
loye_probs=regres.predict(loye_predict)

In [26]:
loye_data["pred"] = loye_probs

In [27]:
loye_data

,Unnamed: 0,following,len_name,num_tweets,screen_name,pred
0,0,1216,11,76808,Feliciaaa_O,0
1,1,675,15,25223,CandaceTheAngel,0
2,2,1779,14,52009,subomiplumptre,0
3,3,674,13,30987,zachyofficial,0
4,4,667,6,36703,rabois,0
5,5,5300,15,19956,_DanielSinclair,0
6,6,762,9,46316,OkiLegend,0
7,7,292,13,3287,MakingCentsPC,0
8,8,101,10,13167,Nia__imani,0
9,9,327,11,129338,brainpicker,0
